This notebook checks if the pytorch lighning early stopping implementation loads back the best weight after the trining has stopped.

In [24]:
import os 
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from torchvision import datasets, transforms
import torch as th
from torch import nn, optim
import torch.nn.functional as F

class LitModel(pl.LightningModule):
    def __init__(self, data):
        super().__init__()
        self.data = data
        self.l1 = nn.Linear(28 * 28, 10)

    def forward(self, x):
        return th.relu(self.l1(x))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        res = self._fake_metric()
        res["loss"] = loss        
        self.log_dict(res, prog_bar=False, logger=False)
        print(early_stopping.wait_count)
        return res

    def _fake_metric(self):
        return {'acc': self.data[self.current_epoch]}

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.02)

In [25]:
import pandas as pd

import mutils

df = pd.read_csv(mutils.csv_dir / "6c6aea26edaa41429b337f389415fcc2" / "version_0" / "metrics.csv")
df = mutils.get_epoch_info(df)

In [26]:
trainer.callback_metrics

{'acc': tensor(0.9015), 'loss': tensor(1.)}

In [27]:
accs = df['valid_1mae'].to_list() # [1,2,3,4,5,6,7,6,5,5,6,4,5,6,5,6,5,6,5,6]
ds_size = 1
train_ds = th.utils.data.TensorDataset(th.rand(ds_size, 28*28), th.ones(ds_size, 10))
train_loader = th.utils.data.DataLoader(train_ds, batch_size=1)
early_stopping = EarlyStopping(monitor="acc", patience=10, verbose=True, mode="max")
trainer = pl.Trainer(callbacks=[early_stopping], max_epochs=len(accs))
model = LitModel(accs)
trainer.fit(model, train_loader)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 7.9 K 
--------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


0



RuntimeError: Early stopping conditioned on metric `acc` which is not available. Pass in or modify your `EarlyStopping` callback to use any of the following: ``